In [ ]:
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
from google.colab import files

BASE_URL = "https://apidadosabertos.saude.gov.br/vigilancia-e-meio-ambiente/sistema-de-informacao-sobre-mortalidade"
MAX_LIMIT = 1000
MAX_WORKERS = 8

def fetch_chunk(offset):
    """Busca um bloco de registros a partir de um offset."""
    params = {"limit": MAX_LIMIT, "offset": offset}
    response = requests.get(BASE_URL, params=params)
    response.raise_for_status()
    data = response.json()
    records = data.get("sim", [])
    print(f"Offset {offset}")
    return records

all_records = []
offset = 0
has_more = True
PASSO = 400000
proxima_meta = PASSO
ultimo_indice_salvo = 0

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = []
    while has_more:
        for _ in range(MAX_WORKERS):
            futures.append(executor.submit(fetch_chunk, offset))
            offset += MAX_LIMIT

        for future in as_completed(futures):
            records = future.result()
            if records:
                all_records.extend(records)

                if len(all_records) >= proxima_meta:
                    novos_registros = all_records[ultimo_indice_salvo:len(all_records)]
                    df_temp = pd.DataFrame(novos_registros)
                    df_temp.to_csv(f"/content/drive/MyDrive/Projeto_Final/mortalidade_{len(all_records)}.csv", index=False, encoding="utf-8")


                    ultimo_indice_salvo = len(all_records)
                    proxima_meta += PASSO
            else:
                has_more = False
        futures.clear()

print(f"Total baixado: {len(all_records)} registros.")

df2 = pd.DataFrame(all_records)

In [ ]:
url_cid = "https://cid10.cpp-ti.com.br/api"
response = requests.get(url_cid)

dados = response.json()

lista_registros = dados['data']
df = pd.DataFrame(lista_registros, columns=["codigo", "nome", "codigo_nome"])

df.to_csv("/content/drive/MyDrive/Projeto_Final/cid10.csv", index=False, encoding="utf-8")